In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master='local', appName='Transformation_and_accions')

# RDD

In [3]:
rdd1 = sc.parallelize([1, 2, 3])
type(rdd1)

pyspark.rdd.RDD

In [4]:
rdd1.collect()

[1, 2, 3]

In [9]:
sc

<SparkContext master=local appName=Transformation_and_accions>

In [7]:
!ls /home/robert/Project/spark-2/transformations_actions/files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista.csv	 evento.csv	      paises.csv
deportista2.csv  juegos.csv	      resultados.csv


In [11]:
path = '/home/robert/Project/spark-2/transformations_actions/files/'

In [12]:
equipos_olimpicos_rdd = sc.textFile(path + 'paises.csv') \
    .map(lambda line : line.split(','))

In [16]:
equipos_olimpicos_rdd.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [21]:
equipos_olimpicos_rdd.map(lambda x: (x[2])).distinct().count()

231

In [23]:
equipos_olimpicos_rdd.map(lambda x: (x[2], x[1])) \
    .groupByKey() \
    .mapValues(len) \
    .take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [24]:
equipos_olimpicos_rdd.map(lambda x: (x[2], x[1])) \
    .groupByKey() \
    .mapValues(list) \
    .take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [29]:
equipos_argentinos = equipos_olimpicos_rdd.filter(lambda l: 'ARG' in l)
equipos_argentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [30]:
equipos_olimpicos_rdd.count()

1185

In [31]:
equipos_olimpicos_rdd.countApprox(20)

1185

In [34]:
deportista_olimpico_rdd = sc.textFile(path + 'deportista.csv') \
    .map(lambda l: l.split(','))

deportista_olimpico_2_rdd = sc.textFile(path + 'deportista2.csv') \
    .map(lambda l: l.split(','))

In [37]:
deportista_olimpico_rdd = deportista_olimpico_rdd \
    .union(deportista_olimpico_2_rdd)

In [39]:
deportista_olimpico_rdd.count()

135572

In [45]:
equipos_olimpicos_rdd.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [43]:
deportista_olimpico_rdd.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [46]:
deportista_olimpico_rdd.map(lambda l: [l[-1], l[:-1]]) \
    .join(equipos_olimpicos_rdd.map(lambda x: [x[0], x[2]])) \
    .take(5)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN'))]

In [47]:
deportista_olimpico_rdd.map(lambda l: [l[-1], l[:-1]]) \
    .join(equipos_olimpicos_rdd.map(lambda x: [x[0], x[2]])) \
    .takeSample(False, 5, 25)

[('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('154', (['39161', 'Angel Merdzhanov Gavrilov', '1', '24', '0', '0'], 'BUL')),
 ('1084',
  (['62843', 'Olha Vasylivna Korobka', '2', '18', '181', '167'], 'UKR')),
 ('678', (['97550', 'Puntsagiin Skhbat', '1', '24', '174', '82'], 'MGL')),
 ('1096', (['106789', 'Hugo Scherzer', '1', '43', '0', '0'], 'USA'))]

In [53]:
resultado = sc.textFile(path + 'resultados.csv') \
    .map(lambda l: l.split(','))

In [54]:
resultado_ganador = resultado.filter(lambda l: 'NA' not in l[1])

In [55]:
resultado_ganador.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [75]:
jugadores_equipos_rdd =deportista_olimpico_rdd.map(lambda l: [l[-1], l[:-1]]) \
    .join(equipos_olimpicos_rdd.map(lambda x: [x[0], x[2]]))

In [79]:
deportista_olimpico_rdd.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [80]:
deportista_paises = deportista_olimpico_rdd \
    .map(lambda l: [l[-1], l[:-1]]) \
    .join(equipos_olimpicos_rdd.map(lambda x: [x[0], x[2]]))

In [82]:
deportista_olimpico_rdd.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [81]:
deportista_paises.take(5)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN'))]

In [83]:
deportista_paises.join(resultado_ganador).take(6)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold')),
 ('74',
  ((['336', 'Arif Yadulla Abdullayev', '1', '27', '164', '60'], 'AZE'),
   'Gold'))]

In [84]:
valores_medallas = {
    'Gold': 7,
    'Silver': 5,
    'Bronze': 4
}

In [85]:
paises_medallas_rdd = deportista_paises.join(resultado_ganador)

In [92]:
paises_puntos = paises_medallas_rdd \
    .map(lambda x: (x[1][0][-1], valores_medallas[x[1][1]]))

In [100]:
paises_puntos.take(10)

[('AZE', 7),
 ('AZE', 7),
 ('AZE', 7),
 ('AZE', 7),
 ('AZE', 7),
 ('AZE', 7),
 ('AZE', 7),
 ('AZE', 7),
 ('AZE', 7),
 ('AZE', 7)]

In [91]:
from operator import add

In [95]:
conclusion = paises_puntos.reduceByKey((add)) \
    .sortBy(lambda x: x[1], ascending=False)

In [99]:
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

In [101]:
sc.stop()